In [1]:
import pandas as pd
import numpy as np
domains=pd.read_csv("/home/fermat/dav/metadata_with_tokens.csv")
domains=domains.replace(np.nan,' ', regex=True)

In [2]:
import pandas as pd
import re
import requests
from bs4 import BeautifulSoup
from requests.exceptions import ConnectionError

requests.packages.urllib3.disable_warnings()
from datetime import datetime
import csv
import sys
import logging
import time

import sys
sys.path.append("/home/fermat/workspace/blackbolt/classifier/classifier/")
from ClassifierModel import ClassifierModel, CategoriesEndpoint, Store2CategoriesMapping

import os
from TopLevelSuggester import CategorySuggester,AbstractionPathSuggester ,NaiveSuggester

ImportError: cannot import name NaiveSuggester

In [3]:
categories_endpoint = 'http://metadata-service.staging-mr.indix.tv:8082/category/all'
categories_provider = CategoriesEndpoint(categories_endpoint)
categories_provider.get_all_categories()
all_categories = categories_provider.categories
suggester = AbstractionPathSuggester(all_categories)
#ns=NaiveSuggester(all_categories)

2016-11-24 10:54:09,836 INFO Loading all categories from http://metadata-service.staging-mr.indix.tv:8082/category/all 
2016-11-24 10:54:33,144 INFO Loaded 10740 categories mapping


In [4]:
domains["tokens"]=domains.domainName.apply(lambda x: [token for token in suggester.vocabulary if token in x ])

domains["bag"]=domains.ix[:,1:-1].apply(lambda x: re.sub(r'[^\x00-\x7F]+',' ',' '.join(x)), axis=1)
domains["categories"]=domains.bag.apply(lambda x :  ns.suggest_categories(x,threshold=0.80))
del domains["Unnamed: 7"]

spotcheck=domains.ix[:,["domainName","categories"]]
spotcheck["domainName"]=spotcheck.domainName.apply(lambda x: "https://"+ x if x.startswith("www") else "https://www."+ x)
spotcheck=spotcheck.drop_duplicates(subset=["domainName"]).dropna()

spotcheck["categorypath1"]=spotcheck.categories.apply(lambda x: [categories_provider.get_top_category_mapping()[t] for t in x ])

spotcheck=spotcheck[spotcheck.categorypath1.apply(lambda x: len(x)>0 and len(x)<8)]

l=spotcheck.apply(lambda x: pd.DataFrame(data = {'categorypath2': x.categorypath1, 'domainName': x.domainName}).to_dict(orient="records"),axis=1)
flat_frame=[item for sublist in l for item in sublist]

df=pd.DataFrame.from_dict(flat_frame)

df=df.replace(np.nan,' ', regex=True)
df["url"]=df.domainName
#df["url"]=df.url.apply(lambda x: "https://"+ x if x.startswith("www") else "https://www."+ x)
df["categorypath1"]=df.apply(lambda x: x.url[8:] + " > " + x.categorypath2 ,axis=1)
df["title"]="None"
df["breadcrumb"]=df.categorypath2
del df["domainName"]
df=df.ix[:,["url","title","categorypath1"]]
df["domain"]=df.url.str[8:]
df.domain=df.domain.str.replace("www.","").apply(lambda x: ".".join(x.split(".")[:-1]))
#df.to_json("spotcheck.json",orient="records")

In [5]:
wasp=pd.read_json("toplevel.txt",orient="index")
wasp.urls=wasp.urls.apply(lambda x: (list(set(["www."+ url for url in x if not url.startswith("www")])) or [""])[0] )
wasp=wasp[wasp.apply(lambda x: len(x.categories)>0 and len(x.urls)>0,axis=1)]
wasp["categories"]=wasp.categories.apply(lambda x: [categories_provider.get_top_category_mapping()[t] for t in x ])

In [6]:
#wasp data testing
del spotcheck["categories"]
spotcheck["key"]=spotcheck.domainName.str.replace("www.","").apply(lambda x: ".".join(x.split(".")[:-1]))
spotcheck["key"]=spotcheck["key"].str[8:]
wasp["key"]=wasp.urls.str.replace("www.","").apply(lambda x: ".".join(x.split(".")[:-1]))
test=wasp.merge(spotcheck, on="key", how='left')
test["actual"]=test.categories
test["predicted"]=test.categorypath1
del test["categories"],test["key"],test["domainName"],test["categorypath1"]
test=test.drop_duplicates(subset=["urls"]).dropna()
test.to_json("wasp_check.json",orient="records")

In [38]:
wasp_dom=wasp.key.unique()
notwasp=df[df.domain.apply(lambda x: x not in wasp_dom)]
del notwasp["domain"]
notwasp.to_json("spotcheck.json",orient="records")

In [ ]:
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import MultiLabelBinarizer
mlb=MultiLabelBinarizer()

import itertools
t=list(set(list(itertools.chain.from_iterable(test.actual))))

mlb.classes=t
predicted=mlb.fit_transform(test.predicted) 
actual=mlb.fit_transform(test.actual)

from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import hamming_loss


In [39]:
hamming_loss(actual,predicted)

0.14384435029410556

In [6]:
wasp_endpoint = os.getenv('WASP_ENDPOINT',
                          'http://wasp.production.indix.tv:9000/configuration?path=production.towncenter.stores.*.{categories,urls}')
store_to_category_mapping = Store2CategoriesMapping(wasp_endpoint)
store_categories = set()
store_category_ids = set(store_to_category_mapping.get_categories("http://www.villeroy-boch.com/shop/toy-s-delight-box-large-20cm.html"))
store_category_ids.update(suggester.suggest_categories(doc="Toy's Delight Large Covered Box 7.75 in" + ' ' + "Tabletop & Decor / Dinnerware / Bowls / Decorative Bowls / Toy's Delight Large Covered Box 7.75 insudo apt-get install winusb


2016-11-24 10:57:48,574 INFO Starting new HTTP connection (1): wasp.production.indix.tv


In [7]:
store_category_ids

{10164, 10168, 10169}

In [10]:
categories_provider.get_top_category_mapping()

{10161: 'computers & accessories',
 10162: 'sports & outdoors',
 10163: 'automotive',
 10164: 'home & kitchen',
 10165: 'office products',
 10166: 'clothing & accessories',
 10167: 'tools & home improvement',
 10168: 'electronics',
 10169: 'jewelry',
 10170: 'health & personal care',
 10171: 'grocery & gourmet food',
 10172: 'toys & games',
 10173: 'arts, crafts & sewing',
 10174: 'baby products',
 10175: 'musical instruments',
 10176: 'patio, lawn & garden',
 10177: 'pet supplies',
 10178: 'industrial & scientific',
 10179: 'shoes',
 10180: 'furniture',
 10181: 'video games',
 10182: 'software',
 27723: 'media',
 99999: 'additional'}